In [1]:
# Data wrangling
import numpy as np
import pandas as pd
from sklearn.preprocessing import FunctionTransformer

# TDA magic
from gtda.mapper import (
    CubicalCover,
    make_mapper_pipeline,
    plot_static_mapper_graph,
    plot_interactive_mapper_graph
)

from sklearn.cluster import DBSCAN

np.random.seed(0)

## Load CSV

In [2]:
# df_original = pd.read_csv("20230726_結果_サンプル.csv")
# objectives= ['f1', 'f2', 'f3', 'f4', 'f5']


# minimize = [
#     True, True, True, # Minimize
#     False, False # Maximize
# ]

# print(df_original.shape)


## Or generate some sample dataset

In [3]:
# Gaussian distributions in n-D

design_variables = ['x0', 'x1']
objectives = ['f1', 'f2']

minimize = [True] * len(objectives)

size = 10000


samples_design_variables = np.random.multivariate_normal(
    mean=[0.0] * len(design_variables),
    cov=np.eye(len(design_variables)), # independent distributions (i.e. eye, aka identity matrix)
    size=size)

samples_objectives = np.random.multivariate_normal(
    mean=[0.0] * len(objectives),
    cov=np.eye(len(objectives)), # independent distributions (i.e. eye, aka identity matrix)
    size=size)

df_design_variables = pd.DataFrame(samples_design_variables, columns= design_variables)
df_objectives       = pd.DataFrame(samples_objectives,       columns= objectives)

df_original = df_design_variables.join(df_objectives)

df_original

,x0,x1,f1,f2
0,1.764052,0.400157,0.330046,-0.000480
1,0.978738,2.240893,0.818116,0.428214
2,1.867558,-0.977278,-2.503947,0.120481
3,0.950088,-0.151357,0.807893,0.602121
4,-0.103219,0.410599,-0.865190,-0.153320
...,...,...,...,...
9995,1.652768,2.018161,-0.259534,-0.000423
9996,0.066968,2.736475,-0.584336,1.006808
9997,-1.012997,0.271662,0.592039,-1.457199
9998,-0.108997,-0.057259,2.011115,1.689858


In [4]:
# Normalize and invert objectives
# Normalization is done because we take the weighted sum of the multiple objectives to build a single objective function,
# which will be passed to the mapper as the single filter function.
# Inversion is done so that everything becomes a minimization problem.

df_objectives_normalized = df_original[objectives]

# Normalize
df_objectives_normalized = (df_objectives_normalized - df_objectives_normalized.min()) / (df_objectives_normalized.max() - df_objectives_normalized.min())

# Invert some rows
for column, column_name in enumerate(df_objectives_normalized):
    if not minimize[column]: # maximize
        df_objectives_normalized[column_name] = 1.0 - df_objectives_normalized[column_name]

In [5]:
def mask_pareto(df_objectives: pd.DataFrame):

    scores = df_objectives.values
    num_points = scores.shape[0]

    pareto_front = np.ones(num_points, dtype=bool)

    for i, i_objectives in enumerate(scores):
        for j, j_objectives in enumerate(scores):
            if i == j: continue

            #  not dominated                     and j dominates
            if all(i_objectives >= j_objectives) and any(i_objectives > j_objectives):
                pareto_front[i] = False
                break # i is not in Pareto front
    return pareto_front


In [6]:
# Select Pareto fronts only; we also reset the index for simplicity
# This is because Giotto-tda's mapper graph (actually igraph.Graph) unfortunately does not take into account pandas' index row.
# (It uses iloc as the row IDs.)
# In other words, we match the pandas' built-in index with iloc so that we don't shoot our own foot.

mask = mask_pareto(df_objectives_normalized)

df_pareto            = df_original[mask].reset_index(drop=True)
df_pareto_objectives = df_objectives_normalized[mask].reset_index(drop = True)

NameError: name 'df_original_objectives' is not defined

## Scatterplot matrix

In [ ]:
import plotly.express as px

fig = px.scatter_matrix(
        df_pareto_objectives
)

fig.update_layout(
    height=1200
)

fig

## Set up mapper 

In [ ]:
# filter_func = np.mean
# color_data  = filter_func(df.values, axis=1)

num_objectives = len(objectives)

# Weights to be used for weighted sum of the objective functions
weights = [1.0 / num_objectives] * num_objectives

# This is the filter function
def weighted_sum(array, axis=1):
    global weights
    return np.dot(array, weights) # dot product (weighted sum)

# Initialise pipeline
pipe = make_mapper_pipeline(
    filter_func=FunctionTransformer( # this transforms an ordinary Python function into a filter function.
        weighted_sum  # actual filter function
    ),
    cover=CubicalCover( # Define cover for mapper
        n_intervals=10,
        overlap_frac=0.45),
    clusterer=DBSCAN(),
    verbose=False,
    n_jobs=1, # parallelism for the clustering step
)

In [ ]:
graph = pipe.fit_transform(df_pareto_objectives)

## Default visualization

In [ ]:
# Use this as the color in the plot
color_data  = weighted_sum(df_pareto_objectives, axis=1)

In [ ]:
plot_static_mapper_graph(
    pipe, df_pareto_objectives, color_data=color_data
)

## With vertical axis as the cluster level

In [ ]:
import igraph
import networkx as nx

def reset_y(graph: nx.Graph, pos, ys): # graph is the networkx graph
    # Apply constraint to y-coordinates
    for node in graph.nodes:
        pos[node][1] = ys[node]


def custom_layout(graph: igraph.Graph, dim):

    ys = { node.index:
            node.attributes()['pullback_set_label']
            for node in graph.vs
    }

    G = nx.Graph()
    for node in graph.vs:
        G.add_node(node.index)
    for edge in graph.get_edgelist():
        G.add_edge(edge[0], edge[1])

    pos = nx.spring_layout(G)
    
    for i in range(10): # improve spring layout by iteration
        # Generate the spring layout positions
        pos = nx.spring_layout(G,
                               pos=pos
                               , k=3.0 # specify the optimal distance between points. Comment out this line to use the default parameter
                              )
        reset_y(G, pos, ys)
    
    return igraph.Layout(
        [
            pos[node]
            for node in G.nodes
        ]
    )

plot_static_mapper_graph(
    pipe, df_pareto_objectives, color_data=color_data,
    layout=custom_layout
)

## Manual Intervention

Find a local minimum in the mapper graph above and set its node as `mapper_node_id` below

In [ ]:
# mapper_node_id = 8
mapper_node_id = 0

# The row IDs in the dataframe. Warning: we don't use the original ID for this, but merely the ID of each Pareto front vertex.
df_row_ids = graph.vs[mapper_node_id].attributes()['node_elements']
df_row_ids

In [ ]:
# Check the contents of the mapper nodes like this:

for v in graph.vs:
    print("mapper node", f"{v.index}:", v.attributes()['node_elements'])

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

knee = df_row_ids

print("Red points: good points (points around the knee point)")

def color(id):
    global knee
    if id in knee:
        return 'red'
    return 'blue'
    

dic = {v: color(v) for v in range(df_objectives.index.max() + 1) }

# Attach marker color in the dataframe
df_colored = df_pareto_objectives.copy(deep=False)

df_colored['index'] = df_colored.index # What we actually want to do is just to create a row with the name 'index'

df_colored['color'] = df_colored['index'].map(dic)

dimensions = []
for each in df_pareto_objectives:
    dimensions.append(
        dict(
            label = each,
            values = df_pareto_objectives[each]
        )
    )

fig = go.Figure(data=go.Splom(
                dimensions=dimensions,
                marker=dict(color=df_colored['color'],
                            showscale=False, # colors encode categorical variables
                           ),
                text=df_colored['index'].map(lambda id: f'id in df: {id}')
                ))

fig.update_layout(
    height=1200
)

for i in range(len(fig.data)):
    fig.data[i].unselected = dict(
        marker=dict(
            color='black',  # unselected points will be black
            opacity=0.3  # with this opacity
        )
    )

fig

In [ ]:
df_pareto.iloc[knee]